#**MEDICAMENTOS VITALES NO DISPONIBLES**


*   Sofia Gomez Rodriguez
*   Victor Andres Martinez Preciado
*   Juan Esteban Paredes Alvarez
*   Sofia Reyes Molina




## 1). Busquen una fuente de datos de tipo estructurados o semi que potencialmente aporte a los ODS.

###The following structured data source was found: https://www.datos.gov.co/Salud-y-Protecci-n-Social/MEDICAMENTOS-VITALES-NO-DISPONIBLES/sdmr-tfmf/about_data

###The dataset found is considered to be directly related to ODS number 3, health and well-being, where the context is about essential and irreplaceable medicines to safeguard the life or alleviate the suffering of a patient or group of patients, which are not available in the country or are not available in sufficient quantities due to low-cost marketing conditions.


##2). Realicen la extracción de datos

In [ ]:
!git clone https://github.com/vamphook972/actividades-etl
%cd actividades-etl

In [ ]:
import pandas as pd

# Read a CSV file delimited by ','
df = pd.read_csv("./data/MEDICAMENTOS_VITALES_NO_DISPONIBLES_20260202.csv")
df.head(5)

## 3). ¿Esta fuente de datos ODS debería cargarse mediante una estrategia de batch o streaming?
###It should be loaded using a batch strategy since a large volume of data is accumulated and it is loaded from a CSV file rather than being processed as it arrives.

## 4) ¿Sería una carga completa o incremental?
###It is a full load because the data is brought from the data source only once, without checking for updates.

## 5) Realicen una exploración del conjunto de datos (revisar nulos, vacios, tamaño del dataset, diccionario, detección de anomalias)

##Dictionary:
https://www.datos.gov.co/Salud-y-Protecci-n-Social/MEDICAMENTOS-VITALES-NO-DISPONIBLES/sdmr-tfmf/about_data

In [ ]:
# Show general information about the dataframe
df.info()

In [ ]:
# This allows you to observe the number of null values per column

serie_1 = df.isna().sum()

for column in serie_1.index:
  por =  100*serie_1[column]/df.shape[0]
  print(f"El porcentaje de nulos en la columna {column}es: {por:.2f}%")

In [ ]:
# Replace empty spaces with null values.
import numpy as np

df = df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# Size of the dataset

print(f"El numero de filas que tiene es de {df.shape[0]} y {df.shape[1]} columnas")

##6). Identificar si la fuente de datos requiere un pre-procesamiento para las siguientes etapas

###The preprocessing steps for the following stages would include verifying the following:

###- Number of negative requests

###- Negative medication concentration values

###- Very old dates

###- Ensuring all dates are in the same format

###- Converting all data fields to their appropriate data types (for example, converting the medication concentration column to float64)

###- Converting the request date to datetime



##7.) Sugerir nuevas fuentes que puedan contextualizar o potenciar los datos ya extraídos
###As new data sources, we can suggest datasets related to high medication prices and mortality caused by the lack of access to vital medicines.

### It would be useful to have the dictionary from the last column of the dataset CODIGO_DIAGNOSTICO_CIE-10, in order to have a better context on the use of each drug and its possible shortage.

# Data Transformation

##In this part, we used a unique to learn more about which units of measurement are available in the DF and whether any need to be standardized.

In [ ]:
df['TIPO_DE_SOLICITUD'].unique()

In [ ]:
df['UNIDAD_MEDIDA1'].unique()

### Delete colum: PRINCIPIO_ACTIVO2

In [ ]:
# We treated the "Not Applicable" values ​​as null
#then decided to analyze what percentage this data represented in the column to see if it was a good option to remove it.
df_cm = df[df["CONCENTRACIÓN_DEL_MEDICAMENTO2"] == "NO APLICA"]
print(f"The percentage of null values ​​in the column CONCENTRACIÓN_DEL_MEDICAMENTO2 is: {(df_cm.shape[0]/df.shape[0])*100:.2f}%")

In [ ]:
# Delete the column: CONCENTRACIÓN_DEL_MEDICAMENTO2
new_df_1 = df.drop('CONCENTRACIÓN_DEL_MEDICAMENTO2', axis=1)

In [ ]:
# The same process as the previous column
df_pa = new_df_1[new_df_1["PRINCIPIO_ACTIVO2"] == "NO APLICA"]
print(f"The percentage of null values ​​in the column PRINCIPIO_ACTIVO2 is: {(df_pa.shape[0]/df.shape[0])*100:.2f}%")

In [ ]:
# Delete the column: PRINCIPIO_ACTIVO2
new_df = df_pa.drop('PRINCIPIO_ACTIVO2', axis=1)

# Delete duplicate registers

In [ ]:
# Delete duplicates registers in the entire df
without_duplicates_df = new_df.drop_duplicates()

In [ ]:
# Count how many duplicates there are in the df
without_duplicates_df.duplicated().sum()

# Coding of "Tipo_solicitud" column

In [ ]:
# convert the "TIPO_DE_SOLICITUD" column to binary
binary = pd.get_dummies(
    without_duplicates_df['TIPO_DE_SOLICITUD'],
    prefix='TIPO_DE_SOLICITUD',
    dtype=int
)

without_duplicates_df = pd.concat([without_duplicates_df, binary], axis=1)
without_duplicates_df

# Standardization of UNIDAD_MEDIDA2 column

In [ ]:
# "µg/ml" and 'mcg/ml' are the same measures
without_duplicates_df['UNIDAD_MEDIDA2'] = (without_duplicates_df['UNIDAD_MEDIDA2'].replace({
        'mg/mL': 'mg/ml',
        'µg/ml': 'mcg/ml'
    }))
without_duplicates_df

# Change datatype of Fecha_autorizacion to datetime

In [ ]:
#converted the format of the FECHA_DE_AUTHORIZATION column into a string
#and then into datatime
#The hour in the entire df is the same, we decided deleted it
without_duplicates_df["FECHA_DE_AUTORIZACIÓN"] = pd.to_datetime(
    without_duplicates_df["FECHA_DE_AUTORIZACIÓN"].astype(str).str.strip(),
    errors="coerce"
).dt.date

without_duplicates_df

# Imputation to null values with median

In [ ]:
# Analize how many records have words and signs
without_duplicates_df['CANTIDAD_SOLICITADA'].astype(str).str.contains(r'[^0-9.]', regex=True).sum()

In [ ]:
# Know how many records are null
without_duplicates_df['CANTIDAD_SOLICITADA'].isnull().sum()

In [ ]:
# Imputation to null values with median
median = without_duplicates_df['CANTIDAD_SOLICITADA'].median()

without_duplicates_df['CANTIDAD_SOLICITADA'] = (
    without_duplicates_df['CANTIDAD_SOLICITADA'].fillna(median))
without_duplicates_df['CANTIDAD_SOLICITADA'].isnull().sum()